# OpenDRIVE - Lanelet2 conversion

### 1. Imports

In [17]:
import os
from pprint import pprint
from lxml import etree
from crdesigner.common.config.lanelet2_config import lanelet2_config
from crdesigner.map_conversion.lanelet2.cr2lanelet import CR2LaneletConverter
from commonroad.scenario.scenario import Location, GeoTransformation
from crdesigner.map_conversion.map_conversion_interface import opendrive_to_commonroad
from pathlib import Path

### 2. Declarations

In [18]:
# Input handling
input_dir = Path("./sample_data")
set_list = [
    "TIERIV_original",
    "CARLA",
    "esmini"
]

# Output handling
output_dir = Path("./output_old")
output_dir.mkdir(exist_ok = True)

# Georeference params
georeference_string = "EPSG:3857"
x_translation = 0.0 # 658761.0
y_translation = 0.0 # 4542599.0

# Scenario handling
location_geotransformation = GeoTransformation(
    georeference_string,
    x_translation = x_translation,
    y_translation = y_translation,
    z_rotation = None,
    scaling = None
)
scenario_location = Location(
    geo_name_id = 11,
    gps_latitude = 0.0,
    gps_longitude = 0.0,
    geo_transformation = location_geotransformation,
    environment = None
)

### 3. Attempt conversion

In [19]:
for set_name in set_list:
    print(f"\n=============== Working on {set_name} ===============\n")
    this_set_input_path = input_dir / set_name
    this_set_output_path = output_dir / set_name
    if not (os.path.exists(this_set_output_path)):
        os.makedirs(this_set_output_path)

    for input_file in os.listdir(this_set_input_path):
        input_file_path = this_set_input_path / input_file
        try:
            # Input handling
            print(f"Converting {input_file_path}")

            # Output handling
            output_filename = f"converted_{input_file}.osm"
            output_path = this_set_output_path / output_filename

            # Scenario initialization
            scenario = opendrive_to_commonroad(input_file_path)
            scenario.location = scenario_location
            
            # Conversion
            if (scenario):
                l2osm = CR2LaneletConverter(lanelet2_config)
                osm = l2osm(scenario)

                with open(output_path, "wb") as file_out:
                    file_out.write(etree.tostring(
                        osm, 
                        xml_declaration = True, 
                        encoding = "UTF-8", 
                        pretty_print = True
                    ))

        except Exception as e:
            print(f"Error: {e}")
            continue


=============== Working on TIERIV_original ===============

Converting sample_data/TIERIV_original/three_straight_lanes.xodr

=============== Working on CARLA ===============

Converting sample_data/CARLA/Town06.xodr
Error: Invalid projection: +lat_0=4.9000000000000000e+1 +lon_0=8.0000000000000000e+0 +type=crs: (Internal Proj Error: proj_create: unrecognized format / unknown name)
Converting sample_data/CARLA/Town05.xodr
Error: Invalid projection: +lat_0=4.9000000000000000e+1 +lon_0=8.0000000000000000e+0 +type=crs: (Internal Proj Error: proj_create: unrecognized format / unknown name)
Converting sample_data/CARLA/Town07.xodr
Error: Invalid projection: +lat_0=4.9000000000000000e+1 +lon_0=8.0000000000000000e+0 +type=crs: (Internal Proj Error: proj_create: unrecognized format / unknown name)
Converting sample_data/CARLA/Town02.xodr
Error: Invalid projection: +lat_0=4.9000000000000000e+1 +lon_0=8.0000000000000000e+0 +type=crs: (Internal Proj Error: proj_create: unrecognized format / unkno

07-Apr-25 23:54:55 - WARNING - ConversionLanelet::lanelet_type: Unknown lane type: {<LaneletType.UNKNOWN: 'unknown'>}
07-Apr-25 23:54:55 - WARNING - ConversionLanelet::lanelet_type: Unknown lane type: {<LaneletType.UNKNOWN: 'unknown'>}
07-Apr-25 23:54:55 - WARNING - ConversionLanelet::lanelet_type: Unknown lane type: {<LaneletType.UNKNOWN: 'unknown'>}
07-Apr-25 23:54:55 - WARNING - ConversionLanelet::lanelet_type: Unknown lane type: {<LaneletType.UNKNOWN: 'unknown'>}
07-Apr-25 23:54:55 - WARNING - OpenDRIVE/traffic_signals.py: Unknown TrafficSignIDGermany of ID 1000003 of subtype -1!
07-Apr-25 23:54:55 - WARNING - OpenDRIVE/traffic_signals.py: Unknown TrafficSignIDGermany of ID 294 of subtype -1!
07-Apr-25 23:54:55 - WARNING - OpenDRIVE/traffic_signals.py: Unknown TrafficSignIDGermany of ID 1000003 of subtype -1!
07-Apr-25 23:54:55 - WARNING - OpenDRIVE/traffic_signals.py: Unknown TrafficSignIDGermany of ID 294 of subtype -1!
07-Apr-25 23:54:55 - WARNING - OpenDRIVE/traffic_signals.py:

Converting sample_data/esmini/multi_intersections.xodr


07-Apr-25 23:54:59 - WARNING - ConversionLanelet::lanelet_type: Unknown lane type: {<LaneletType.UNKNOWN: 'unknown'>}
07-Apr-25 23:54:59 - WARNING - ConversionLanelet::lanelet_type: Unknown lane type: {<LaneletType.UNKNOWN: 'unknown'>}
/home/tranhuunhathuy/anaconda3/envs/llconv/lib/python3.9/site-packages/crdesigner/map_conversion/common/conversion_lanelet_network.py:942: UserWarning: For traffic light with ID 47 no referencing lanelet was found!
  warnings.warn(
/home/tranhuunhathuy/anaconda3/envs/llconv/lib/python3.9/site-packages/crdesigner/map_conversion/common/conversion_lanelet_network.py:942: UserWarning: For traffic light with ID 48 no referencing lanelet was found!
  warnings.warn(
/home/tranhuunhathuy/anaconda3/envs/llconv/lib/python3.9/site-packages/crdesigner/map_conversion/common/conversion_lanelet_network.py:942: UserWarning: For traffic light with ID 49 no referencing lanelet was found!
  warnings.warn(
/home/tranhuunhathuy/anaconda3/envs/llconv/lib/python3.9/site-packag

Converting sample_data/esmini/striaghtAndCurves.xodr
Converting sample_data/esmini/e6mini-lht.xodr


/home/tranhuunhathuy/anaconda3/envs/llconv/lib/python3.9/site-packages/crdesigner/map_conversion/common/utils.py:74: UserWarning: geoidgrids removed from projection string
  warnings.warn("geoidgrids removed from projection string")
07-Apr-25 23:55:02 - WARNING - Value guide-post is not a supported road object type!
07-Apr-25 23:55:02 - WARNING - Value guide-post is not a supported road object type!


Converting sample_data/esmini/straight_500m_signs.xodr
Error: 'SWE'
Converting sample_data/esmini/curve_r100.xodr
Converting sample_data/esmini/soderleden.xodr
Error: int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Converting sample_data/esmini/straight_500m_roadmarks.xodr


/home/tranhuunhathuy/anaconda3/envs/llconv/lib/python3.9/site-packages/crdesigner/map_conversion/common/utils.py:74: UserWarning: geoidgrids removed from projection string
  warnings.warn("geoidgrids removed from projection string")
/home/tranhuunhathuy/anaconda3/envs/llconv/lib/python3.9/site-packages/crdesigner/map_conversion/common/utils.py:74: UserWarning: geoidgrids removed from projection string
  warnings.warn("geoidgrids removed from projection string")
07-Apr-25 23:55:03 - WARNING - Value rail-pole is not a supported road object type!
07-Apr-25 23:55:03 - WARNING - Value rail-pole is not a supported road object type!
07-Apr-25 23:55:03 - WARNING - Value guide-post is not a supported road object type!
07-Apr-25 23:55:03 - WARNING - Value guide-post is not a supported road object type!


Converting sample_data/esmini/e6mini.xodr
Converting sample_data/esmini/fabriksgatan.xodr
Converting sample_data/esmini/crest-curve.xodr
Converting sample_data/esmini/jolengatan.xodr
Converting sample_data/esmini/straight_500m.xodr


/home/tranhuunhathuy/anaconda3/envs/llconv/lib/python3.9/site-packages/crdesigner/map_conversion/common/utils.py:74: UserWarning: geoidgrids removed from projection string
  warnings.warn("geoidgrids removed from projection string")
